In [1]:
#Import Libraries
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing
import wrangle as w
import env

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import wrangle as w

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = w.get_mall_customers()

df.head()


,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


NameError: name 'df_summary' is not defined